# Comparision between classical Matrix Multiplication Algorithm and Tiled Matrix Multiplication

In this practical work we will compare these two algorithms, and check how the memory access impacts in the performance.

**Authors of the homework:**
+ Mireia Izquierdo
+ Aleksandra Jamróz

These 2 cells will help us to know which hardware we have assigned, to decide the block size in our parallel algorithm.

Sometimes the 256 threads per block (16x16 threads) are the best compromise solution.

In [6]:
#Check that runtime type is set to GPU !!
#Uncomment the follow line if you are running in Google Colaboratory
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 6.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 8.2 MB/s 
     |████████████████████████████████| 70 kB 9.4 MB/s 
  Created wheel for pycuda: filename=pycuda-2022.1-cp37-cp37m-linux_x86_64.whl size=629701 sha256=d43abd1a8a3c3a9902e6837347f18de002ba24bfe7fe1d656f2a0d313242f7b1
  Stored in directory: /root/.cache/pip/wheels/17/53/c9/caa05618e686df51f017d8a9923f38d915ce31df67ab6628e6
  Created wheel for pytools: filename=pytools-2022.1.12-py2.py3-none-any.whl size=65034 sha256=491b40c619e67bb2e99a92a27bafe6437679b52ab0ba9287c108cb76955b7648
  Stored in directory: /root/.cache/pip/wheels/37/5e/9e/76d7430e116b7cab0016fbabb26b896daae1946a3f7dea9915
Successfully built pycuda pytools


In [7]:
#Import the necessary modules, such as pycuda, numpy and time.
import  numpy  as  np  
import  pycuda.autoinit #Contains all functions that are automatically imported when using pycuda
from    pycuda.compiler import SourceModule
import  pycuda.driver as  drv #Python wrapper for CUDA
import  pycuda.gpuarray as  gpuarray
import  pycuda.tools as gtools
from numpy import linalg as la #NumPy linear algebra functions
import time

# FIRST KERNEL: simple matrices multiplication
This kernel will recieve 3 matrices: a and b are the source matrices, and c is where we will store the result.

The size of the matrix a is $m *n$.

The size of the matrix b is $n * o$.

The size of the matrix c is $m * o$.

All three matrices are stored in a row-wise vector. 

Each thread will have assigned one cell position in the matrix c. The formulae for both coordinates are:


$idxY = blockIdx.y*blockDim.y+threadIdx.y$

$idxX = blockIdx.x*blockDim.x+threadIdx.x$



In [8]:
kernel  =  SourceModule ("""
__global__ void matrix_mult(float* a, 
                            float* b, 
                            float* c, 
                            int m, 
                            int n, 
                            int o) 
{ 
  // a: vector which represents the matrix_a
  // b: vector which represents the matrix_b
  // c: vector where we will store the resulting matrix
  // m: number of rows of matrix a
  // n: number of columns of matrix a, and number of rows of matrix b
  // o: number of columns of matrix b

  // First task: Using threadIdx.x, threadIdx.y, blockDim.x, blockDim.y, blockIdx.x, blockDim.y identify the coordinates x and y in the result matrix
  // Implement the matrix multiplication (cell by cell) using the conventional code.

  int idxX;
  int idxY;
  int idxZ;
  int offA;

  float s;

  idxY = blockIdx.y * blockDim.y + threadIdx.y; //Calculate the row address in target matrix
  idxX = blockIdx.x * blockDim.x + threadIdx.x; //Calculate the column address in target matrix


  if ( idxX < o && idxY < m ){    //Verify the row address and column address are valid
    idxZ = idxY*o + idxX;         //Calculate the target vector address, 
                                  //Assuming it is a row-wise matrix representation
    s = 0;                        //Initialize the s acumulator

    offA = idxY * n;                //Calculate the offset of row, in matrix a row-wise representation
                                   //Reduce the number of calculae in the next for loop

    for ( int i = 0; i<n; i++)     //Run through the a columns, b rows
      s += a[offA+i] * b[(i*o) + idxX];
    
    c[idxZ] = s;
  }
  
}
""")

In [9]:
#Get the CUDA function matrix_mult and assign a Python reference.
matrix_mult = kernel.get_function("matrix_mult")

#print matrix_mult
#print(matrix_mult)
#output: <pycuda._driver.Function object at 0x7fd8313a4f80>

For the first case, we creates 3 matrices of size 1024 * 1024 (this is for a fair play comparation with the tiled matrix multiplication).

In [10]:
#define the number of rows and columns for matrix A and B (i.e. the sizes of matrices A and B)
#we choose size 1024 * 1024 for a fair comparision with the tiled matrix multiplication
numRowsA = 1024
numColsA = 1024
numRowsB = 1024
numColsB = 1024

#Create 3 matrices. A and B are filled with random numbers, C is filled with 0s
matrix_a = np.random.randn(numRowsA,numColsA).astype(np.float32)
matrix_b = np.random.randn(numRowsB,numColsB).astype(np.float32)
matrix_c = np.zeros((numRowsA,numColsB),dtype = np.float32)

In [11]:
#Upload the matrices to the GPU, i.e. compile the matrices into GPU arrays
matrix_a_gpu = gpuarray.to_gpu(matrix_a)
matrix_b_gpu = gpuarray.to_gpu(matrix_b)
matrix_c_gpu = gpuarray.to_gpu(matrix_c)

Most GPUs have a multiple core number of 256. This allow us to have the maximum number of blox in execution.

In [12]:
#Define a block execution size of 16x16x1 threads. It will allocate 256 threads per block < 1024 (the most common maximum threads/block)
block_size = (16,16,1) #We take this value to get up to 256 parallel threads per block 
                     #This will allow us to get up to 9 parallel blocks in execution in a K80
                     #10 parallel blocks in execution in a T4
                     #14 parallel blocks in execition in a GTX 1080 Ti
                     #17 parallel blocks in execition in a RTX 2080 Ti

In [13]:
#Calculate the number of blocks needed in the x and y axis
numblocks_x = int(np.ceil(numColsB/16)) 
numblocks_y = int(np.ceil(numRowsA/16))

In [14]:
#Define the grid size
grid_size = (numblocks_x,numblocks_y)

In [15]:
#Execute the matrices multiplication algorithm & calculate how long it takes to run
start_t = time.time()
matrix_mult(matrix_a_gpu,
            matrix_b_gpu,
            matrix_c_gpu,
            np.int32(numRowsA),
            np.int32(numColsA),
            np.int32(numColsB),
            block = block_size,
            grid = (numblocks_x,numblocks_y))
end_t = time.time()

In [16]:
#Calculate how long it took to run
print("Time processing: {0} seconds".format(end_t-start_t))
#Time processing: 0.0021321773529052734 seconds

Time processing: 0.002038240432739258 seconds


In [17]:
#Get the C result matrix to compare it with the locally computed result
matrix_c_final = matrix_c_gpu.get()

In [18]:
#multiply matrix a with matrix b
d = np.matmul(matrix_a,matrix_b)

In [19]:
#compute and print the MSE
MSE = np.sum(np.sum(np.power(matrix_c_final-d,2)))/(d.shape[0]*d.shape[1])
print("Mean Square Error: {0}".format(MSE))
#Mean Square Error: 3.1641275710647676e-10

Mean Square Error: 3.501988699028402e-10


In [20]:
#Returns the norm of the matrix 
la.norm(matrix_c_final - d)
#output: 0.018214824

0.019162565

In [21]:
matrix_c_final[290:299,90:99]

array([[-19.214125  ,  28.685335  ,   2.4757004 ,  13.44657   ,
          6.88626   ,  15.903372  ,   0.91665703,  21.422142  ,
         39.52686   ],
       [-28.007004  ,  -2.9562702 ,  28.780758  , -66.84069   ,
        -27.194145  ,   3.9199438 ,  21.470493  ,  23.766273  ,
        -34.02734   ],
       [-43.53081   ,  25.132624  ,  38.90423   ,  53.428616  ,
         -8.998992  ,  21.043875  ,  -6.9387746 ,  57.83198   ,
         37.003414  ],
       [-55.544216  ,   7.955236  ,  -3.6870196 ,  27.92066   ,
        -13.283571  , -30.683647  , -55.629448  ,  20.763615  ,
         -3.9933147 ],
       [ -3.5327082 ,  30.961285  ,  22.20616   , -20.47816   ,
        -14.204374  , -48.756508  ,  29.608973  ,  36.26625   ,
          4.3091226 ],
       [-32.4318    ,  85.255684  , -20.408857  ,   6.906548  ,
          2.591298  ,   5.0773463 ,  10.055532  , -20.074034  ,
        -44.83915   ],
       [-48.336933  , -11.225642  ,  44.524017  , -18.938278  ,
         62.335537  ,  11.1581

In [22]:
d[290:299,90:99]

array([[-19.214117  ,  28.685352  ,   2.4757013 ,  13.446564  ,
          6.88624   ,  15.903359  ,   0.91666627,  21.42212   ,
         39.526848  ],
       [-28.00703   ,  -2.9562678 ,  28.780758  , -66.84068   ,
        -27.19414   ,   3.9199324 ,  21.470488  ,  23.766273  ,
        -34.02735   ],
       [-43.53081   ,  25.13264   ,  38.904217  ,  53.42858   ,
         -8.999003  ,  21.043861  ,  -6.9387646 ,  57.832     ,
         37.003384  ],
       [-55.544216  ,   7.9552298 ,  -3.6869965 ,  27.920687  ,
        -13.283572  , -30.683653  , -55.629467  ,  20.763641  ,
         -3.993318  ],
       [ -3.5326986 ,  30.961262  ,  22.20615   , -20.478157  ,
        -14.204379  , -48.756496  ,  29.608967  ,  36.266235  ,
          4.3091245 ],
       [-32.431797  ,  85.25569   , -20.40885   ,   6.906542  ,
          2.5912971 ,   5.0773535 ,  10.055547  , -20.074045  ,
        -44.83918   ],
       [-48.336952  , -11.225642  ,  44.524033  , -18.938276  ,
         62.335533  ,  11.1581

# Analysis
The Problem with the previous code is the memory access is no coalesced.<br>

![image.png](attachment:f31c927a-80fe-4bd4-90ac-074ff2f0f00a.png)


So, there is an algorithm which allows an smart memory handling, using the shared memory tiling: 

In [23]:
kernel_code_template = """
__global__ void MatrixMulKernel(float *A, float *B, float *C)
{

  const uint wA = %(MATRIX_A_SIZE)s;
  const uint wB = %(MATRIX_B_SIZE)s;  

  // Block index
  const uint bx = blockIdx.x;
  const uint by = blockIdx.y;

  // Thread index
  const uint tx = threadIdx.x;
  const uint ty = threadIdx.y;

  // Index of the first sub-matrix of A processed by the block const uint aBegin = wA * %(BLOCK_SIZE)s * by;
  // Index of the last sub-matrix of A processed by the block const uint aEnd = aBegin + wA - 1;
  // Step size used to iterate through the sub-matrices of A const uint aStep = %(BLOCK_SIZE)s;

  // Index of the first sub-matrix of B processed by the block const uint bBegin = %(BLOCK_SIZE)s * bx;
  // Step size used to iterate through the sub-matrices of B const uint bStep = %(BLOCK_SIZE)s * wB;

  // The element of the block sub-matrix that is computed by the thread
  float Csub = 0;

  // Loop over all the sub-matrices of A and B required to compute the block sub-matrix
  for (int a = aBegin, b = bBegin;
       a <= aEnd;
       a += aStep, b += bStep) 
    {
      // Shared memory for the sub-matrix of A
      __shared__ float As[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];
      // Shared memory for the sub-matrix of B
      __shared__ float Bs[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];

      // Load the matrices from global memory to shared memory. Each thread loads one element of each matrix
      As[ty][tx] = A[a + wA * ty + tx];
      Bs[ty][tx] = B[b + wB * ty + tx];
      
      // Sync to make sure the matrices are loaded
      __syncthreads();

      // Multiply the two matrices together; each thread computes one element of the block sub-matrix
      for (int k = 0; k < %(BLOCK_SIZE)s; ++k)
        Csub += As[ty][k] * Bs[k][tx];

      // Sync to make sure that the preceding computation is done before loading two new sub-matrices of A and B in the next iteration
      __syncthreads();
    }

  // Write the block sub-matrix to global memory. Each thread writes one element const uint c = wB * %(BLOCK_SIZE)s * by + %(BLOCK_SIZE)s * bx;
  C[c + wB * ty + tx] = Csub;
}
"""


In [24]:
#Define the (square) matrix size
MATRIX_A_SIZE = 1024
MATRIX_B_SIZE = 1024

#Define size of blocks & tiles sub-matrix (we assume that the block size is same as tile size)
TILE_SIZE = 16
BLOCK_SIZE = TILE_SIZE

This is a trick to replace in our original code the constants MATRIX_A_SIZE, MATRIX_B_SIZE and BLOCK_SIZE for their values.

It allows us change the matrix sizes without changing the original code.

(It is not CUDA trick, it's a Python trick using strings and % parameters.

In [25]:
#Get the kernel code from the template by specifying the constants MATRIX_A_SIZE, MATRIX_B_SIZE, and BLOCK_SIZE
kernel_code = kernel_code_template % { 
    'MATRIX_A_SIZE': MATRIX_A_SIZE,
    'MATRIX_B_SIZE': MATRIX_B_SIZE,
    'BLOCK_SIZE': BLOCK_SIZE
}

In [ ]:
#Compile the kernel code, getting the kernel function reference, in matrixmul.
mod = SourceModule(kernel_code)

In [28]:
#Create two random square matrices
a_cpu = matrix_a
b_cpu = matrix_b

#Compute reference on the CPU to verify GPU computation
c_cpu = np.dot(a_cpu, b_cpu)

Instead of creating a host matrix c with 0s, we can reserve the memory in the GPU. Afterwards we can copy the results from the memory to a new local variable.<br/>
This will reduce the amount of data to upload (and thus the time it needs).

In [29]:
#Transfer host (CPU) memory to device (GPU) memory 
a_gpu = gpuarray.to_gpu(a_cpu) 
b_gpu = gpuarray.to_gpu(b_cpu)

#Create empty GPU array for the result (C = A * B)
c_gpu = gpuarray.empty((MATRIX_A_SIZE, MATRIX_B_SIZE), np.float32)

In [ ]:
#Get the kernel function from the compiled module
matrixmul = mod.get_function("MatrixMulKernel")

In [31]:
#Define the block size
block_size = (int(TILE_SIZE),int(TILE_SIZE),1)

#Define the grid size
grid_size = (int(np.ceil(MATRIX_A_SIZE/TILE_SIZE)),int(np.ceil(MATRIX_A_SIZE/TILE_SIZE)))

Invokes the matrices multiplication kernel, and measure the timming.

In [33]:
a = np.int32(1024)
b = np.int32(1024)
c = np.int32(1024)

#Call the kernel on the card
start_t = time.time()
matrixmul(
    # inputs
    a_gpu, b_gpu, 
    # output
    c_gpu, 
    # grid of multiple blocks
    grid = grid_size,
    # block of multiple threads
    block = block_size 
    )
end_t = time.time()

NameError: ignored

In [ ]:
print("Time processing: {0} seconds".format(end_t-start_t))
#Time processing: 0.0002009868621826172 seconds

Time processing: 0.0002009868621826172 seconds


In [ ]:
c_cpu_final = c_gpu.get()
c_cpu_final

In [ ]:
new_d = np.matmul(matrix_a,matrix_b)
new_d

In [ ]:
MSE = np.sum(np.sum(np.power(c_cpu_final-d,2)))/(d.shape[0]*d.shape[1])
print("Mean Square Error: {0}".format(MSE))
#Mean Square Error: 3.1641275710647676e-10

# PRACTICAL WORK:

Modify the previous code to recieive any dimensional matrices, and be able to multipy them using tiled memory.

Take care about boundaries in the final matrix, and fill with zeroes the memory places where the original matrices are not defined.

In [34]:
kernel_code_various_matrix_mult = """
__global__ void various_matrix_mult(float *A, float *B, float *C, int m, int n, int o)
{

// Shared memory for the sub-matrix of A
__shared__ float ds_A[%(TILE_SIZE)s][%(TILE_SIZE)s];
// Shared memory for the sub-matrix of B
__shared__ float ds_B[%(TILE_SIZE)s][%(TILE_SIZE)s];


// Thread index
const uint tx = threadIdx.x;
const uint ty = threadIdx.y;

int row = blockIdx.y * %(TILE_SIZE)s + threadIdx.y;
int col = blockIdx.x * %(TILE_SIZE)s + threadIdx.x;

// The element of the block sub-matrix that is computed by the thread
float Csub = 0;

for (int l = 0; l < ceil((float)n/%(TILE_SIZE)s) ; ++l) {     //iterate through tiles

  // Load the matrices from global memory to shared memory, each thread loads one element of each matrix
  // blank spaces filling with zeros:

  if (row < m  && l * %(TILE_SIZE)s + tx < n)
    ds_A[ty][tx] = A[row *n  + l * %(TILE_SIZE)s + tx];
  else
    ds_A[ty][tx] = 0.0;

  if (l * %(TILE_SIZE)s + ty < n  && col < o)
    ds_B[ty][tx] = B[(l * %(TILE_SIZE)s + ty) *o  + col];
  else
    ds_B[ty][tx] = 0.0;  

  // Synchronize to make sure the matrices are loaded
  __syncthreads();  

  // Multiply the two matrices together; each thread computes one element of the block sub-matrix
  for (int k = 0; k < %(TILE_SIZE)s  &&  k < n ; ++k) {
      Csub +=  ds_A[ty][k] * ds_B[k][tx];
  }

  // Synchronize to make sure that the preceding computation is done before loading two new sub-matrices of A and B in the next iteration
    __syncthreads();
}

if (row < m  && col < o)
  C[row*o+col] = Csub;

}
"""

In [35]:
# Define tile size
TILE_SIZE = 16

# Get the kernel code from the template by specifying the constant
kernel_code = kernel_code_various_matrix_mult % { 'TILE_SIZE': TILE_SIZE }

In [37]:
#Compile the kernel code & get the kernel function
mod = SourceModule(kernel_code)
various_matrix_mult = mod.get_function("various_matrix_mult")

In [38]:
#Define the shape of the matrices
m = 1023     #Number of rows of matrix a 
n = 1025     #Number of columns of matrix a and rows of matrix b
o = 1027    #Number of columns of matrix b

In [39]:
#Create random input matrices and output matrix, all as dtype float 
matrix_a = np.random.randn(m,n).astype(np.float32)
matrix_b = np.random.randn(n,o).astype(np.float32)
matrix_c = np.zeros((m,o),dtype = np.float32)

In [40]:
#Transfer host (CPU) memory to device (GPU) memory 
matrix_a_gpu = gpuarray.to_gpu(matrix_a)
matrix_b_gpu = gpuarray.to_gpu(matrix_b)
matrix_c_gpu = gpuarray.to_gpu(matrix_c)

In [41]:
#Define block size
block_size = (16,16,1) # We take this value to get up to 256 parallel threads per block 
                     # This will allow us to get up to 9 parallel blocks in execution in a K80
                     # 10 parallel blocks in execution in a T4
                     # 14 parallel blocks in execition in a GTX 1080 Ti
                     # 17 parallel blocks in execition in a RTX 2080 Ti

In [42]:
numblocks_x = int(np.ceil(o/16))
numblocks_y = int(np.ceil(m/16))
grid_size = (numblocks_x, numblocks_y)

In [43]:
#Invoke the matrix multiplication kernel and count the time it takes to run it
start_t = time.time()
various_matrix_mult(matrix_a_gpu,
                    matrix_b_gpu,
                    matrix_c_gpu,
                    np.int32(m),
                    np.int32(n),
                    np.int32(o),
                    block = block_size,
                    grid = (numblocks_x,numblocks_y))
end_t = time.time()

In [44]:
#Print the time it takes to process this
print("Time processing: {0} seconds".format(end_t - start_t))

Time processing: 0.0003409385681152344 seconds


In [45]:
#Get the result matrix from GPU memory
c_cpu_final = matrix_c_gpu.get()
c_cpu_final

array([[-19.612833 ,  -9.863956 , -19.060343 , ..., -51.75292  ,
        -84.98523  ,  20.660486 ],
       [ -1.8771812,   1.1504304, -23.739288 , ..., -49.818176 ,
         14.193877 ,  -6.9832363],
       [-24.945776 , -51.99384  , -26.073706 , ..., -32.01726  ,
         13.040907 ,  -5.4235473],
       ...,
       [-15.770132 ,  12.6268   ,  23.238705 , ...,  28.151834 ,
         51.06493  , -14.744515 ],
       [ 15.833657 , -36.64549  ,   7.1878767, ...,  -7.38496  ,
          5.566458 ,  -6.062813 ],
       [-10.538173 ,  19.447533 , -17.217821 , ...,  24.515432 ,
        -10.96253  ,  46.84473  ]], dtype=float32)

In [46]:
#Compare matmul with numpy
compare_matrix = np.matmul(matrix_a,matrix_b)
compare_matrix

array([[-19.612814 ,  -9.863983 , -19.060352 , ..., -51.752903 ,
        -84.98523  ,  20.660473 ],
       [ -1.8771677,   1.1504226, -23.739292 , ..., -49.8182   ,
         14.193876 ,  -6.983244 ],
       [-24.945787 , -51.993904 , -26.073715 , ..., -32.01727  ,
         13.04091  ,  -5.4235544],
       ...,
       [-15.770135 ,  12.626825 ,  23.238697 , ...,  28.151836 ,
         51.06491  , -14.744509 ],
       [ 15.833665 , -36.645477 ,   7.1878633, ...,  -7.384941 ,
          5.566455 ,  -6.0628157],
       [-10.538168 ,  19.447529 , -17.217812 , ...,  24.515423 ,
        -10.962535 ,  46.84471  ]], dtype=float32)

In [47]:
#Calculate and print the MSE
MSE = np.sum(np.sum(np.power(c_cpu_final-compare_matrix,2)))/(compare_matrix.shape[0] * compare_matrix.shape[1])
print("Mean Square Error: {0}".format(MSE))
#Mean Square Error: 3.500120241522478e-10

Mean Square Error: 3.500120241522478e-10


# Conclusion
Matrix multiplication is a good example of a process that can be performed more efficiently by using parallel computing. 

By tiling, we can reduce the global memory accesses by taking advantage of the GPU's shared memory. When we partition the data into subsets that fit into the shared memory, we can really exploit the advantaged of memory-level parallelism.

This can be clearly seen in the improvement in runtime (efficiency), when comparing the different methods. 